In [1]:
from pathlib import PosixPath, Path
import SimpleITK as sitk
import itkwidgets
import nibabel as nib
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import os
import numpy as np
import json
from typing import List

In [2]:
flag_env = load_dotenv(
    os.path.join("/".join(os.getcwd().split("/")[:-2]), "envs/abdomenct_seg.env")
)
if flag_env:
    root_dir = Path(os.environ.get("ROOT_DIR"))
else:
    raise Exception(f"Failed to load the environment file!")

In [3]:
ct_img = sitk.ReadImage(root_dir / "imagesTr" / "FLARE22_Tr_0001_0000.nii.gz")
seg_img = sitk.ReadImage(root_dir / "labelsTr" / "FLARE22_Tr_0001.nii.gz")

### Reference:
- https://github.com/SimpleITK/SimpleITK/issues/561
- https://github.com/InsightSoftwareConsortium/SimpleITK-Notebooks/blob/master/Python/21_Transforms_and_Resampling.ipynb
- https://gist.github.com/mrajchl/ccbd5ed12eb68e0c1afc5da116af614a

In [4]:
out_spacing = [1.5, 1.5, 1.5]

In [9]:
def resample_sitk(
    img: sitk.Image, target_resolution: List = [1.5, 1.5, 1.5], is_label: bool = False
):
    """
    Given a sitk.image, resample to the given out_spacing.
    Output Spacing based on monai's ct bundle recommendatin. This  may vary based on model usage and requirements. Say a vessel with 0.5 resolution is expected!

    """

    original_spacing = img.GetSpacing()
    original_size = img.GetSize()
    out_size = [
        int(np.round(original_size[0] * (original_spacing[0] / out_spacing[0]))),
        int(np.round(original_size[1] * (original_spacing[1] / out_spacing[1]))),
        int(np.round(original_size[2] * (original_spacing[2] / out_spacing[2]))),
    ]

    resampler = sitk.ResampleImageFilter()
    resampler.SetOutputSpacing(target_resolution)
    resampler.SetSize(out_size)
    resampler.SetOutputDirection(img.GetDirection())
    resampler.SetOutputOrigin(img.GetOrigin())
    resampler.SetTransform(sitk.Transform())
    resampler.SetDefaultPixelValue(img.GetPixelIDValue())

    if is_label:
        resampler.SetInterpolator(sitk.sitkLabelLinear)
    else:
        resampler.SetInterpolator(sitk.sitkLinear)
    ##
    return resampler.Execute(img)

In [10]:
ct_imgV2 = resample_sitk(img=ct_img, target_resolution=[1.5, 1.5, 2.0], is_label=False)

seg_imgV2 = resample_sitk(
    img=seg_img, target_resolution=[1.5, 1.5, 2.0], is_label=False
)

In [11]:
itkwidgets.view(image=ct_img, label_image=seg_img)

Viewer(geometries=[], gradient_opacity=0.22, interpolation=False, point_sets=[], rendered_image=<itk.itkImageP…

In [12]:
itkwidgets.view(image=ct_imgV2, label_image=seg_imgV2)

Viewer(geometries=[], gradient_opacity=0.22, interpolation=False, point_sets=[], rendered_image=<itk.itkImageP…

## Observation:
* Resampling the labelmap add a lot of noise at the edge. May be wise to dilate by 1. This is important to reduce the noise in the labels!
* Playing with the window-levels and cross-referencing radiopedia for window level, the intensity range [-50, 450] captures all the regions of interest.